### Python Intermediate Practice Code

#### Intro to Data Cleaning

Data with a consistent format is often described as "clean." As data scientists, not all data we encounter is clean; we often need to prepare it in a process called __data cleaning__.

We'll perform data cleaning on a real-world data set of artworks contained in the Museum of Modern Art (MoMA).